# Para el Dataset de KOI

In [9]:
with open(data_path, "r") as f:
    for i in range(5):
        print(f.readline())

# This file was produced by the NASA Exoplanet Archive  http://exoplanetarchive.ipac.caltech.edu

# Wed Oct  1 11:42:42 2025

#

# User preference: *

#



In [10]:
import pandas as pd

data_path = "../Data/cumulative_2025.10.01_11.42.42.csv"

# Intentar con distintos separadores
try:
    df = pd.read_csv(data_path, sep=",", low_memory=False)
except:
    try:
        df = pd.read_csv(data_path, sep="\t", low_memory=False)
    except:
        df = pd.read_csv(data_path, sep=";", low_memory=False)

# 1. Dimensiones
print("Dimensiones del dataset:", df.shape)

# 2. Primeras filas
print("\nPrimeras filas del dataset:")
print(df.head())

# 3. Información del dataset
print("\nInformación del dataset:")
print(df.info())

# 4. Valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# 5. Porcentaje de nulos
print("\nPorcentaje de nulos por columna:")
print((df.isnull().mean() * 100).round(2))

# 6. Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df.describe())

Dimensiones del dataset: (9710, 1)

Primeras filas del dataset:
  # This file was produced by the NASA Exoplanet Archive  http://exoplanetarchive.ipac.caltech.edu
0                         # Wed Oct  1 11:42:42 2025                                              
1                                                  #                                              
2                               # User preference: *                                              
3                                                  #                                              
4                     # COLUMN kepid:          KepID                                              

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9710 entries, 0 to 9709
Data columns (total 1 columns):
 #   Column                                                                                            Non-Null Count  Dtype 
---  ------                                                                         

Correción!:

In [ ]:
# Lo haremos bien, pandas el archivo no es un CSV directo, sino que 
# viene con un montón de metadatos y comentarios al inicio (líneas que empiezan con #).
# Por eso, Pandas lo está leyendo todo como una sola columna, 
# en lugar de reconocer las columnas de datos.

import pandas as pd

data_path = "../Data/cumulative_2025.10.01_11.42.42.csv"

# Cargar dataset ignorando las líneas de comentarios
df = pd.read_csv(data_path, comment="#")

print("Dimensiones del dataset:", df.shape)

print("\nColumnas:")
print(df.columns)

print("\nPrimeras filas:")
print(df.head())

print("\nInformación general:")
print(df.info())

print("\nValores nulos por columna:")
print(df.isnull().sum().sort_values(ascending=False).head(10))  # Top 10 con más nulos

print("\nPorcentaje de nulos por columna:")
print((df.isnull().mean() * 100).round(2).sort_values(ascending=False).head(10))

print("\nEstadísticas descriptivas:")
print(df.describe(include="all"))

Dimensiones del dataset: (9564, 141)

Columnas:
Index(['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition',
       'koi_vet_stat', 'koi_vet_date', 'koi_pdisposition', 'koi_score',
       'koi_fpflag_nt',
       ...
       'koi_dicco_mdec', 'koi_dicco_mdec_err', 'koi_dicco_msky',
       'koi_dicco_msky_err', 'koi_dikco_mra', 'koi_dikco_mra_err',
       'koi_dikco_mdec', 'koi_dikco_mdec_err', 'koi_dikco_msky',
       'koi_dikco_msky_err'],
      dtype='object', length=141)

Primeras filas:
   rowid     kepid kepoi_name   kepler_name koi_disposition koi_vet_stat  \
0      1  10797460  K00752.01  Kepler-227 b       CONFIRMED         Done   
1      2  10797460  K00752.02  Kepler-227 c       CONFIRMED         Done   
2      3  10811496  K00753.01           NaN       CANDIDATE         Done   
3      4  10848459  K00754.01           NaN  FALSE POSITIVE         Done   
4      5  10854555  K00755.01  Kepler-664 b       CONFIRMED         Done   

  koi_vet_date koi_pdisposition  koi_

In [12]:
# Determinemos cosas en base a los datos nulos
# Para eso: ordenaremos columnas por porcentaje de valores nulos
nulls = (df.isnull().mean() * 100).sort_values(ascending=False)

print("\nTop 20 columnas con más valores nulos:")
print(nulls.head(20))

print("\nColumnas completamente llenas (sin nulos):")
print(nulls[nulls == 0].index.tolist())


Top 20 columnas con más valores nulos:
koi_incl_err1       100.000000
koi_sma_err2        100.000000
koi_sma_err1        100.000000
koi_teq_err1        100.000000
koi_teq_err2        100.000000
koi_longp_err2      100.000000
koi_ingress         100.000000
koi_ingress_err2    100.000000
koi_ingress_err1    100.000000
koi_longp_err1      100.000000
koi_longp           100.000000
koi_eccen_err2      100.000000
koi_eccen_err1      100.000000
koi_sage_err1       100.000000
koi_sage_err2       100.000000
koi_sage            100.000000
koi_model_dof       100.000000
koi_model_chisq     100.000000
koi_incl_err2       100.000000
kepler_name          71.277708
dtype: float64

Columnas completamente llenas (sin nulos):
['rowid', 'kepid', 'kepoi_name', 'koi_vet_stat', 'koi_disposition', 'koi_fpflag_ec', 'koi_fpflag_co', 'koi_fpflag_ss', 'koi_fpflag_nt', 'koi_pdisposition', 'koi_vet_date', 'koi_period', 'koi_duration', 'koi_count', 'koi_parm_prov', 'koi_fittype', 'koi_time0bk', 'koi_time0', 'koi_d

## De aquí podemos decir que: 

### Columnas problemáticas:

Hay muchísimas columnas con 100% de nulos (koi_incl_err1, koi_sma_err1/2, koi_teq_err1/2, koi_longp, koi_eccen_err1/2, koi_sage, koi_model_dof, etc.). Esas columnas son inservibles para el modelado → podemos descartarlas de inmediato.


### Columnas útiles (0% nulos, completas):

Estas son muy buenas candidatas:

*Identificadores:* rowid, kepid, kepoi_name → no aportan nada predictivo, así que se descartan.

*Labels / targets:*

1. koi_disposition (target principal).

2. koi_pdisposition (clasificación preliminar, útil para comparar).

3. Flags: koi_fpflag_ec, koi_fpflag_co, koi_fpflag_ss, koi_fpflag_nt → indicadores booleanos muy relevantes.

*Orbitales y físicos clave:*

1. koi_period (período orbital).

2. koi_duration (duración del tránsito).

3. koi_time0bk, koi_time0 (época del tránsito, quizás menos útiles, pero podrían ayudar).

4. ra, dec (coordenadas celestes, más informativo para localización, no tanto para clasificación).

5. Extras: koi_vet_stat, koi_vet_date, koi_disp_prov, koi_count, koi_parm_prov, koi_fittype → más metadatos del proceso de validación.


## En base a esto, procedemos de la siguiente manera: 

Podemos ahora dividir las columnas en tres grupos:

### Target

koi_disposition, porque es lo que queremos predecir.

### Features candidatas (input del modelo)

1. Físicas: koi_period, koi_duration, koi_prad, koi_teq, koi_insol, koi_srad, koi_steff, koi_depth, koi_model_snr.

2. Flags: koi_fpflag_nt, koi_fpflag_ss, koi_fpflag_co, koi_fpflag_ec.

3. Score: koi_score.

### Descartables (de entrada):

1. Identificadores (rowid, kepid, kepoi_name).
2. Columnas con 100% nulos.
3. Fechas (koi_vet_date) y cosas administrativas (koi_parm_prov, koi_disp_prov), salvo que quieras usarlas para auditoría.

In [13]:
# Vamos a limpiar el dataset y quedarnos con las features útiles + target:

# Eliminar columnas con 100% nulos
df_clean = df.dropna(axis=1, how="all")

# Eliminar columnas de identificadores
drop_cols = ["rowid", "kepid", "kepoi_name", "kepler_name"]
df_clean = df_clean.drop(columns=[c for c in drop_cols if c in df_clean.columns])

# Mantener target y features candidatas
target = "koi_disposition"

features = [
    "koi_period", "koi_duration", "koi_prad", "koi_teq", "koi_insol",
    "koi_srad", "koi_steff", "koi_depth", "koi_model_snr", "koi_score",
    "koi_fpflag_nt", "koi_fpflag_ss", "koi_fpflag_co", "koi_fpflag_ec"
]

df_model = df_clean[[c for c in features if c in df_clean.columns] + [target]]

print("Dimensiones del dataset limpio:", df_model.shape)
print("Columnas seleccionadas:")
print(df_model.columns)
print("\nValores nulos restantes:")
print(df_model.isnull().sum())

Dimensiones del dataset limpio: (9564, 15)
Columnas seleccionadas:
Index(['koi_period', 'koi_duration', 'koi_prad', 'koi_teq', 'koi_insol',
       'koi_srad', 'koi_steff', 'koi_depth', 'koi_model_snr', 'koi_score',
       'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_disposition'],
      dtype='object')

Valores nulos restantes:
koi_period            0
koi_duration          0
koi_prad            363
koi_teq             363
koi_insol           321
koi_srad            363
koi_steff           363
koi_depth           363
koi_model_snr       363
koi_score          1510
koi_fpflag_nt         0
koi_fpflag_ss         0
koi_fpflag_co         0
koi_fpflag_ec         0
koi_disposition       0
dtype: int64


In [14]:
# Ahora vamos a guardar el dataset limpio en un nuevo archivo CSV

import os

# Ruta de la nueva carpeta
output_dir = "../FilteredData"
os.makedirs(output_dir, exist_ok=True)

# Ruta completa del archivo
output_path = os.path.join(output_dir, "KOI_Filtrated.csv")

# Guardar CSV
df_model.to_csv(output_path, index=False)

print(f"Archivo guardado en: {output_path}")


Archivo guardado en: ../FilteredData/KOI_Filtrated.csv
